In [1]:
%load_ext autoreload
%autoreload 2
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
import peft
import torch
torch.backends.cuda.matmul.allow_tf32 = True
import torch.nn as nn
import transformers
from utils import get_local_dir, get_local_run_dir, disable_dropout, init_distributed, get_open_port
import os
import hydra
import torch.multiprocessing as mp
from omegaconf import OmegaConf, DictConfig
import trainers
import wandb
import json
import socket
from typing import Optional, Set
import resource
from hnet.lora import LoRA_controller, LoRA
from transformers import LlamaModel, LlamaConfig,LlamaForCausalLM
from hnet.hypernet import HyperNetController, HyperNet, HyperNetLinear,PolicyWrapper
from accelerate import Accelerator
from hnet.lora import LoRA_controller, LoRA


os.environ['TRANSFORMERS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_DATASETS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['TORCH_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
cache_dir = '/home/mila/e/emiliano.penaloza/scratch/models'


/home/mila/e/emiliano.penaloza/rlhf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
policy = transformers.AutoModelForCausalLM.from_pretrained(
    'vicgalle/gpt2-open-instruct-v1', cache_dir=cache_dir, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map=None).to('cuda')
base_policy = transformers.AutoModelForCausalLM.from_pretrained(
    'vicgalle/gpt2-open-instruct-v1', cache_dir=cache_dir, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map=None).to('cuda')
tokenizer = transformers.AutoTokenizer.from_pretrained(
    'vicgalle/gpt2-open-instruct-v1', cache_dir=cache_dir, use_fast=True)
policy = policy.to('cuda')
hypernet = HyperNet(16, 0, 64,16,10,64,10, 10,10,10)



In [3]:
from omegaconf import DictConfig, OmegaConf
configs_d = {'model': {'target_modules':['c_attn']},
             'hnet':{'r':16}}
config = OmegaConf.create(configs_d)


In [4]:
x = tokenizer("hello world", return_tensors='pt', padding='max_length', max_length=1024, truncation=True)
x = {k: v.to('cuda') for k, v in x.items()}
policy(**x)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[  4.3320,   4.2891,   1.4424,  ...,   3.6055,   4.3867,   4.7578],
         [  2.4395,   4.3594,  -0.3579,  ...,  -7.6445,  -3.6582,  -2.9473],
         [-14.0156,  -9.8828, -12.2969,  ..., -16.9531, -14.9062, -15.5781],
         ...,
         [ -3.5781,  -3.4590,  -5.9727,  ...,  -3.4199,  -2.9629,  -3.0371],
         [ -3.6172,  -3.5020,  -6.0117,  ...,  -3.4453,  -2.9980,  -3.0703],
         [ -3.2852,  -3.1465,  -5.6836,  ...,  -3.2520,  -2.7402,  -2.8086]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<UnsafeViewBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [5]:
controller = LoRA_controller(config)
controller.augmentLLM(policy)
controller.freezeParams(policy, False)


key='transformer.h.0.attn.c_attn'
key='transformer.h.1.attn.c_attn'
key='transformer.h.2.attn.c_attn'
key='transformer.h.3.attn.c_attn'
key='transformer.h.4.attn.c_attn'
key='transformer.h.5.attn.c_attn'
key='transformer.h.6.attn.c_attn'
key='transformer.h.7.attn.c_attn'
key='transformer.h.8.attn.c_attn'
key='transformer.h.9.attn.c_attn'
key='transformer.h.10.attn.c_attn'
key='transformer.h.11.attn.c_attn'


In [6]:

base_policy(**x)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[  4.3320,   4.2891,   1.4424,  ...,   3.6055,   4.3867,   4.7578],
         [  2.4395,   4.3594,  -0.3579,  ...,  -7.6445,  -3.6582,  -2.9473],
         [-14.0156,  -9.8828, -12.2969,  ..., -16.9531, -14.9062, -15.5781],
         ...,
         [ -3.5781,  -3.4590,  -5.9727,  ...,  -3.4199,  -2.9629,  -3.0371],
         [ -3.6172,  -3.5020,  -6.0117,  ...,  -3.4453,  -2.9980,  -3.0703],
         [ -3.2852,  -3.1465,  -5.6836,  ...,  -3.2520,  -2.7402,  -2.8086]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<UnsafeViewBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [10]:
controller.change_forward('adaptor')
policy(**x)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[  4.3320,   4.2891,   1.4424,  ...,   3.6055,   4.3867,   4.7578],
         [  2.4395,   4.3594,  -0.3579,  ...,  -7.6445,  -3.6582,  -2.9473],
         [-14.0156,  -9.8828, -12.2969,  ..., -16.9531, -14.9062, -15.5781],
         ...,
         [ -3.5781,  -3.4590,  -5.9727,  ...,  -3.4199,  -2.9629,  -3.0371],
         [ -3.6172,  -3.5020,  -6.0117,  ...,  -3.4453,  -2.9980,  -3.0703],
         [ -3.2852,  -3.1465,  -5.6836,  ...,  -3.2520,  -2.7402,  -2.8086]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<UnsafeViewBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [ ]:
hypernet = HyperNet(16, 0, 64,16,10,64,10, 10,10,10)



In [15]:
controller.change_forward('base')

In [2]:
%load_ext autoreload
%autoreload 2
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_DATASETS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['TORCH_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
cache_dir = '/home/mila/e/emiliano.penaloza/scratch/models'

In [8]:
import torch
b= torch.zeros(10,10,requires_grad=True)
a =torch.ones(10,10,requires_grad=True)



In [9]:
(a@b).sum().backward()

In [11]:
b.grad

tensor([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.]])

In [7]:
import pickle as pkl 
with open('/home/mila/e/emiliano.penaloza/direct-preference-optimization/notebooks/data/train_data.pkl','rb') as f:
    train_data = pkl.load(f)

In [9]:
import pickle as pkl 
with open('/home/mila/e/emiliano.penaloza/direct-preference-optimization/notebooks/data/test_data.pkl','rb') as f:
    test_data = pkl.load(f)

In [11]:
import json 
with open('/home/mila/e/emiliano.penaloza/direct-preference-optimization/notebooks/data/test_data.json','r') as f:
    test_json = json.load(f)
print(f"{len(test_data)=}")


JSONDecodeError: Extra data: line 2 column 1 (char 989)

In [5]:

print(f"{len(train_data)=}")

len(train_data)=19959


In [27]:
conv1d = model.transformer.h[0].attn.c_attn

In [35]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [32]:
from torch.nn import Conv1d
#give me a sample forward 
import torch
import torch.nn.functional as F

tensor = torch.randn(10, 15, 768)

output = conv1d(tensor)

In [33]:
output.shape

torch.Size([10, 15, 2304])

In [1]:
import torch
import torch.nn as nn
#load a transformer encoder 
from transformers import BertModel, BertTokenizer
encoder = nn.TransformerEncoderLayer(d_model=768, nhead=12)
encoder_full = nn.TransformerEncoder(encoder, num_layers=12)

/home/mila/e/emiliano.penaloza/rlhf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder_full

TransformerEncoder(
  (layers): ModuleList(
    (0-11): 12 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (linear1): Linear(in_features=768, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=768, bias=True)
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
)

In [1]:
import torch.multiprocessing as mp
import torch
import resource

def foo(x):
    print(f"{x=}")
if __name__ == '__main__':

    world_size = torch.cuda.device_count()
    print('starting', world_size, 'processes for FSDP training')
    soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
    resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))
    print(f'setting RLIMIT_NOFILE soft limit to {hard} from {soft}')
    mp.set_start_method('forkserver')
    mp.spawn(foo, nprocs=world_size, args=(world_size), join=True)

starting 4 processes for FSDP training
setting RLIMIT_NOFILE soft limit to 1048576 from 1048576


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/cvmfs/ai.mila.quebec/apps/arch/distro/python/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/cvmfs/ai.mila.quebec/apps/arch/distro/python/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'foo' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/cvmfs/ai.mila.quebec/apps/arch/distro/python/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/cvmfs/ai.mila.quebec/apps/arch/distro/python/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'foo' on <module '__main__' (built-in)>
Traceback (most recent call last):

ProcessExitedException: process 2 terminated with exit code 1

In [18]:
import torch
from torch.nn import MultiheadAttention
from transformers import LlamaModel, LlamaConfig,LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import torch.nn as nn 
from argparse import ArgumentParser
from functools import partial
#if the directory is not the root of the project, change it
os.chdir('/home/mila/e/emiliano.penaloza/RLPHF')
import LoRA.loralib  as lora
from notebooks.loss import *
import pandas as pd 
os.environ['TRANSFORMERS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_DATASETS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['TORCH_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
cache_dir = '/home/mila/e/emiliano.penaloza/scratch/models'



/home/mila/e/emiliano.penaloza/rlhf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:

device = 'cpu'
# Initializing a LLaMA llama-7b style configuration
configuration = LlamaConfig()
print(f"{configuration=}")

configuration.num_hidden_layers = 2
configuration.num_attention_heads = 16
configuration.num_key_value_heads = 2
tokenizer = AutoTokenizer.from_pretrained("allenai/tulu-2-dpo-13b",cache_dir = cache_dir)
model = LlamaForCausalLM(configuration).to(device)





configuration=LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.39.2",
  "use_cache": true,
  "vocab_size": 32000
}



In [11]:
if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id

In [19]:
#create an argument parser 
parser = ArgumentParser()
parser.add_argument("--loss", type=str, help="path to the model")
parser.add_argument("--max_length", type=int, help="path to the model")
args = parser.parse_args([])

In [20]:
#load json data /home/mila/e/emiliano.penaloza/RLPHF/notebooks/data/train_data.json
import json

# Initialize an empty list to store the JSON objects
data = []

# Open the file and read the lines
with open('/home/mila/e/emiliano.penaloza/RLPHF/notebooks/data/train_data.json', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append it to the list
        if line:
            try:
                # Parse each line as a JSON object and append it to the list
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}\nError: {e}")

df = pd.DataFrame(data)


In [14]:
df.if_chosen.isna().sum()

0

In [21]:
#import display from ipython
from IPython.display import display
dfs_uid = df.groupby('user_id')
processed_data = []
for i,(uid, df_uid) in enumerate(dfs_uid):
    df_utt = df_uid.groupby('interaction_id')
    for j,(utt, df_utt) in enumerate(df_utt):

        if len(df_utt) >=  2:
            try:
                if len(df_utt[df_utt['if_chosen'] == True]) >= 2:
                    #take the one with the higher score 
                    df_score_1 = df_utt.score.values[0]
                    df_score_2 = df_utt.score.values[1]
                    if df_score_1 > df_score_2:
                        chosen = df_utt['if_chosen'].values[0]
                        non_chosen = df_utt['if_chosen'].values[1]
                    elif df_score_1 < df_score_2:
                        chosen = df_utt['if_chosen'].values[1]
                        non_chosen = df_utt['if_chosen'].values[0]

                    else:
                 
                        continue
                else:
                    chosen = df_utt[df_utt['if_chosen'] == True].model_response.values[0]
                    non_chosen = df_utt[df_utt['if_chosen'] == False].model_response.values[0]
                # print(f"{df_utt[df_utt['if_chosen'] == False].model_response=}")


                user_prompt = df_utt['user_prompt'].values[0]
                user_id = df_utt['user_id'].values[0]
                processed_data.append([user_id, user_prompt, chosen, non_chosen])
            except:
                print(f"{df_utt[df_utt['if_chosen'] == False].model_response=}")
                display(df_utt)
                print('error')
                raise Exception
        else:
            print('no match')
        continue
    # if i == 10:
    #         print(f"{processed_data=}")
    #         print(f"{len(processed_data)=}")
            

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
# device = 'cpu'
class CustomDataset(Dataset):
    def __init__(self, processed_data):
        self.data = processed_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out = {}
        out['uid'] = self.data[idx][0]
        out['prompt'] = self.data[idx][1]
        out['chosen'] = self.data[idx][2]
        out['rejected'] = self.data[idx][3]
        return out

# Create a CustomDataset instance
dataset = CustomDataset(processed_data)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True,collate_fn=partial(collate_fn, tokenizer=tokenizer, max_length=args.max_length, device=device))

In [28]:
for b in dataloader:
    print(f"{b.keys()=}")

    #tokenize non_chosen and preffered 
    model_output = model(input_ids=b['prompt_prefered_ids'], attention_mask=b['prompt_prefered_mask'])
    
    break
    
    
    

b.keys()=dict_keys(['uids', 'prompt_prefered_ids', 'prompt_disprefered_ids', 'prompt_prefered_mask', 'prompt_disprefered_mask', 'labels_prefered', 'labels_disprefered'])


In [30]:
# Initialize an empty list to store the JSON objects
#/home/mila/e/emiliano.penaloza/RLPHF/notebooks/data/{split}_data.csv does not exits
split = 'test'
import json
import pandas as pd 
data = []

# Open the file and read the lines

with open(f'/home/mila/e/emiliano.penaloza/RLPHF/notebooks/data/{split}_data.json', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append it to the list
        if line:
            try:
                # Parse each line as a JSON object and append it to the list
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}\nError: {e}")

df = pd.DataFrame(data)
dfs_uid = df.groupby('user_id')
processed_data = []
for i,(uid, df_uid) in enumerate(dfs_uid):
    df_utt = df_uid.groupby('interaction_id')
    for j,(utt, df_utt) in enumerate(df_utt):

        if len(df_utt) >=  2:
            try:
                if len(df_utt[df_utt['if_chosen'] == True]) >= 2:
                    #take the one with the higher score 
                    df_score_1 = df_utt.score.values[0]
                    df_score_2 = df_utt.score.values[1]
                    if df_score_1 > df_score_2:
                        chosen = df_utt.model_response.values[0]
                        non_chosen = df_utt.model_response.values[1]
                    elif df_score_1 < df_score_2:
                        chosen = df_utt.model_response.values[1]
                        non_chosen = df_utt.model_response.values[0]

                    else:
                
                        continue
                else:
                    chosen = df_utt[df_utt['if_chosen'] == True].model_response.values[0]
                    non_chosen = df_utt[df_utt['if_chosen'] == False].model_response.values[0]
                # print(f"{df_utt[df_utt['if_chosen'] == False].model_response=}")


                user_prompt = df_utt['user_prompt'].values[0]
                user_id = df_utt['user_id'].values[0]
                processed_data.append([user_id, user_prompt, chosen, non_chosen])
            except:
                print(f"{df_utt[df_utt['if_chosen'] == False].model_response=}")

                print('error')
                raise Exception
        else:
            print('no match')
        continue




no match
no match
no match
no match
no match


In [31]:
for example in processed_data:

        prompt = '\n\n Human: ' + example[1] + "\n\n Assistant:"
        if example[1] == "who else do you think represents the most threat to them in the nfc?":
            print(f"{example[1]=}")
            responses = [example[2], example[3]]
            print(responses)
            raise Exception

example[1]='who else do you think represents the most threat to them in the nfc?'
['The biggest threats to the Cowboys in the NFC are likely the Eagles, 49ers, and Buccaneers. The Eagles have a dynamic offense and are the defending NFC East champs. The 49ers have a formidable defense and made it to the NFC Championship last season. And the Buccaneers still have Tom Brady leading a talented roster. Getting through any of those teams in the playoffs would be a serious challenge for Dallas.', 'The biggest threats to the Cowboys in the NFC are likely the Eagles, 49ers, and Buccaneers. The Eagles have a dynamic offense and are the defending NFC East champs. The 49ers have a formidable defense and made it to the NFC Championship last season. And the Buccaneers still have Tom Brady leading a talented roster. Getting through any of those teams in the playoffs would be a serious challenge for Dallas.']


Exception: 

In [16]:
df[df.model_response == True ]

,utterance_id,interaction_id,conversation_id,user_id,turn,within_turn_id,conversation_type,user_prompt,model_response,model_name,model_provider,score,if_chosen,included_in_balanced_subset


In [29]:
df[df.user_prompt ==  "who else do you think represents the most threat to them in the nfc?"]

,utterance_id,interaction_id,conversation_id,user_id,turn,within_turn_id,conversation_type,user_prompt,model_response,model_name,model_provider,score,if_chosen,included_in_balanced_subset
7935,ut23731,int6608,c2743,user521,1,0,unguided,who else do you think represents the most thre...,The biggest threats to the Cowboys in the NFC ...,claude-2.1,anthropic,9,True,True
7936,ut23732,int6608,c2743,user521,1,1,unguided,who else do you think represents the most thre...,The biggest threats to the Cowboys in the NFC ...,claude-2.1,anthropic,6,True,True


In [29]:
b['labels_prefered']

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     1, 10604, 29901,
           450,  2315,   297,   278,  5057, 15194,   338,  3625,   304,  4840,
           373, 16631, 15512, 13987,   297,  8314, 29889,   887,   508,  1804,
           701,   363,   263,  3889, 14260,   470, 2

In [37]:
model_output.logits.shape

torch.Size([32, 408, 32000])

In [32]:
b['prompt_prefered_ids'].shape

torch.Size([32, 640])

In [39]:
b['prompt_prefered_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
b['prompt_prefered_mask'][1]

NameError: name 'b' is not defined